# #Cài đặt PYSPARK

In [1]:
!pip install pyspark
!pip install findspark
!pip install translate==3.6.1

In [2]:
import findspark
findspark.init()

from pyspark.sql import SparkSession

spark = SparkSession.builder.appName("Jobs Dataframe").getOrCreate()

spark

In [3]:
from google.colab import drive # kết nối lên drive
drive.mount('/content/drive')

Mounted at /content/drive


# UPDATE FILE LÊN TỪ DRIVE

In [4]:
import pandas as pd
df = pd.read_csv("/content/drive/MyDrive/full_dataset.csv")
df = spark.createDataFrame(df)

In [5]:
# Hiển thị schema ban đầu để kiểm tra
print("Schema ban đầu:")
df.printSchema()

# Hiển thị vài dòng đầu để xem dữ liệu
print("\nDữ liệu ban đầu:")
df.show(5, truncate=False)

Schema ban đầu:
root
 |-- ten_cong_viec: string (nullable = true)
 |-- khu_vuc: string (nullable = true)
 |-- ngay_dang: string (nullable = true)
 |-- link: string (nullable = true)
 |-- cap_bac: string (nullable = true)
 |-- kinh_nghiem: string (nullable = true)
 |-- mo_ta_cong_viec: string (nullable = true)
 |-- yeu_cau_cong_viec: string (nullable = true)
 |-- nganh_nghe: string (nullable = true)


Dữ liệu ban đầu:
+-----------------------------------------------------------------------------------------------------------------------------------------+-----------+----------+-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+----------------------+-----------+--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

# **TIỀN XỬ LÝ DỮ LIỆU**

In [6]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, lower, regexp_replace, trim, when, lit, coalesce,isnan, length
from pyspark.sql.types import IntegerType, StringType
import re
from pyspark.ml.feature import Tokenizer, StopWordsRemover
import pandas as pd

In [7]:
# Tạo điều kiện lọc tổng hợp
condition = None
for column_name in df.columns:
    # Điều kiện cho từng cột: nếu cột này có giá trị xấu
    col_condition = (
        col(column_name).isNull() |                  # Kiểm tra NULL
        isnan(col(column_name)) |                    # Kiểm tra NaN (kiểu float)
        (trim(col(column_name)) == "") |             # Chuỗi rỗng sau khi trim
        (length(trim(col(column_name))) == 0) |      # Độ dài = 0 sau khi trim
        (col(column_name) == "nan") |                # Chuỗi "nan"
        (col(column_name) == "NaN") |                # Chuỗi "NaN"
        (col(column_name).cast("string").rlike("^\\s*$"))  # Chỉ chứa khoảng trắng
    )
    # bắt được là các cột này có những dữ liệu xấu nào
    # Kết hợp điều kiện giữa các cột
    if condition is None:
        condition = col_condition
    else:
        condition = condition | col_condition # A hoặc B -> (A hoặc B) hoặc C -> ((A hoặc B) hoặc C ) hoặc D -> gộp các điều kiện với nhau

# Lọc các dòng KHÔNG thỏa mãn bất kỳ điều kiện xấu nào
df_clean = df.filter(~condition)
df_clean.show(5,truncate=False)
print("Số dòng trong DataFrame ban đầu:", df.count())
print("Số dòng trong DataFrame sau khi làm sạch:", df_clean.count())

+-----------------------------------------------------------------------------------------------------------------------------------------+-----------+----------+-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+----------------------+-----------+---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

# 1.  Chuẩn hóa ngay_dang (ĐÃ XONG)


In [8]:
from pyspark.sql.functions import col, to_date, when, lit, trim
from pyspark.sql.types import DateType, TimestampType

In [9]:
from os import truncate

df = df_clean.withColumn("ngay_dang_clean", when(
    (trim(col("ngay_dang")).isNull())|
    (trim(col("ngay_dang")) == "")|
    (trim(col("ngay_dang")).rlike("(?i)^nan$")),
    None
    ).otherwise(trim(col("ngay_dang")))
)
#rlike("(?i)^nan$") kiểm tra xem một chuỗi có đúng bằng "nan" hay không, không phân biệt chữ hoa hay thường.

#Đổi format thường gặp sang Date
df = df.withColumn(
    "ngay_dang_date",
    coalesce(
        to_date(col("ngay_dang_clean"), "d/M/yyyy"),
        to_date(col("ngay_dang_clean"), "dd/MM/yyyy"),
        to_date(col("ngay_dang_clean"), "d-MM-yyyy"),
        to_date(col("ngay_dang_clean"), "dd-MM-yyyy")
    )
)
# coalesce(...) thử nhiều định dạng ngày khác nhau để không bị mất dữ liệu do format không khớp.
df.select("ngay_dang", "ngay_dang_clean", "ngay_dang_date").show(10,truncate=False)

distinct_ngay_dang = df.select("ngay_dang_date").distinct()

+----------+---------------+--------------+
|ngay_dang |ngay_dang_clean|ngay_dang_date|
+----------+---------------+--------------+
|18-07-2025|18-07-2025     |2025-07-18    |
|18-07-2025|18-07-2025     |2025-07-18    |
|18-07-2025|18-07-2025     |2025-07-18    |
|18-07-2025|18-07-2025     |2025-07-18    |
|18-07-2025|18-07-2025     |2025-07-18    |
|18-07-2025|18-07-2025     |2025-07-18    |
|18-07-2025|18-07-2025     |2025-07-18    |
|18-07-2025|18-07-2025     |2025-07-18    |
|18-07-2025|18-07-2025     |2025-07-18    |
|18-07-2025|18-07-2025     |2025-07-18    |
+----------+---------------+--------------+
only showing top 10 rows



In [10]:
distinct_ngay_dang.orderBy("ngay_dang_date", ascending=True).show(50,truncate=False)

+--------------+
|ngay_dang_date|
+--------------+
|2025-06-18    |
|2025-06-19    |
|2025-06-20    |
|2025-06-21    |
|2025-06-23    |
|2025-06-24    |
|2025-06-25    |
|2025-06-26    |
|2025-06-27    |
|2025-06-28    |
|2025-06-29    |
|2025-06-30    |
|2025-07-01    |
|2025-07-02    |
|2025-07-03    |
|2025-07-04    |
|2025-07-05    |
|2025-07-06    |
|2025-07-07    |
|2025-07-08    |
|2025-07-09    |
|2025-07-10    |
|2025-07-11    |
|2025-07-12    |
|2025-07-13    |
|2025-07-14    |
|2025-07-15    |
|2025-07-16    |
|2025-07-17    |
|2025-07-18    |
+--------------+



In [11]:
#Ghi giá trị date chuẩn hóa về cột chính ngay_dang
#df = df.withColumn("ngay_dang", col("ngay_dang_date"))

#Xóa cột trung gian ngay_dang_date nếu không cần nữa
#df = df.drop("ngay_dang_date")
#df = df.drop("ngay_dang_clean")

df.show(5, truncate=False)
print("Số dòng trong DataFrame sau khi làm sạch:", df.count())

+-----------------------------------------------------------------------------------------------------------------------------------------+-----------+----------+-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+----------------------+-----------+---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

# Chuẩn hóa link (ĐÃ XONG)

In [12]:
from pyspark.sql.functions import col, trim

df = df.withColumn("link", trim(col("link")))
df = df.withColumn(
    "link", when(
        (col("link").isNull()) |
        (col("link") == "") |
        (col("link") == "nan"),
        None
    ).otherwise(col("link"))
)
df = df.withColumn("is_valid_link", col("link").rlike("^(http|https)://"))

# Đếm số lượng link hợp lệ
valid_link_count = df.filter(col("is_valid_link") == True).count()
print("Số lượng link hợp lệ:", valid_link_count)


Số lượng link hợp lệ: 11290


In [13]:
#df = df.drop("is_valid_link")
df.show(10, truncate=False)

+-----------------------------------------------------------------------------------------------------------------------------------------+-----------+----------+-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+----------------------+-----------+---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

#Chuẩn hóa cap_bac(ĐÃ XONG)


1. Kiểm tra có bao nhiêu loại cấp bậc

In [14]:
# Lấy danh sách các giá trị riêng biệt trong cột cap_bac
distinct_cap_bac = df.select("cap_bac").distinct()

# Đếm số loại cap_bac riêng biệt
num_types = distinct_cap_bac.count()
print(f"Tổng số loại cap_bac: {num_types}")

# In ra danh sách các giá trị cap_bac
distinct_cap_bac.show(truncate=False)

Tổng số loại cap_bac: 12
+------------------------+
|cap_bac                 |
+------------------------+
|Tổng giám đốc           |
|Giám đốc                |
|Phó Giám đốc            |
|Mới tốt nghiệp          |
|Quản lý                 |
|Nhân viên               |
|Trưởng nhóm / Giám sát  |
|Thực tập sinh/Sinh viên |
|Sinh viên/ Thực tập sinh|
|Mới Tốt Nghiệp          |
|Giám Đốc và Cấp Cao Hơn |
|Trưởng phòng            |
+------------------------+



2. Chuẩn hóa

In [15]:
from pyspark.sql.functions import col, trim, lower, when

# Đưa về: chữ thường, loại bỏ khoảng trắng đầu/cuối
df = df.withColumn("cap_bac", trim(lower(col("cap_bac"))))

# Chuẩn hóa các giá trị tương tự
df = df.withColumn("cap_bac_standardized", when(
    col("cap_bac").isin("mới tốt nghiệp", "mới tốt nghiệp "), "mới tốt nghiệp"
).when(
    col("cap_bac").isin("thực tập sinh/sinh viên", "sinh viên/ thực tập sinh"), "thực tập sinh/sinh viên"
).when(
    col("cap_bac").isin("giám đốc và cấp cao hơn", "giám đốc"), "giám đốc"
).otherwise(col("cap_bac")))

# Gán mã cho từng cấp bậc
df = df.withColumn("cap_bac_code", when(col("cap_bac_standardized") == "tổng giám đốc", 1)
    .when(col("cap_bac_standardized") == "giám đốc", 2)
    .when(col("cap_bac_standardized") == "phó giám đốc", 3)
    .when(col("cap_bac_standardized") == "trưởng phòng", 4)
    .when(col("cap_bac_standardized") == "quản lý", 5)
    .when(col("cap_bac_standardized") == "trưởng nhóm / giám sát", 6)
    .when(col("cap_bac_standardized") == "nhân viên", 7)
    .when(col("cap_bac_standardized") == "thực tập sinh/sinh viên", 8)
    .when(col("cap_bac_standardized") == "mới tốt nghiệp", 9)
    .otherwise(-1)
)
# Kiểm tra danh sách chuẩn hóa
df.select("cap_bac", "cap_bac_standardized", "cap_bac_code").distinct().orderBy("cap_bac_code").show(truncate=False)

+------------------------+-----------------------+------------+
|cap_bac                 |cap_bac_standardized   |cap_bac_code|
+------------------------+-----------------------+------------+
|tổng giám đốc           |tổng giám đốc          |1           |
|giám đốc                |giám đốc               |2           |
|giám đốc và cấp cao hơn |giám đốc               |2           |
|phó giám đốc            |phó giám đốc           |3           |
|trưởng phòng            |trưởng phòng           |4           |
|quản lý                 |quản lý                |5           |
|trưởng nhóm / giám sát  |trưởng nhóm / giám sát |6           |
|nhân viên               |nhân viên              |7           |
|thực tập sinh/sinh viên |thực tập sinh/sinh viên|8           |
|sinh viên/ thực tập sinh|thực tập sinh/sinh viên|8           |
|mới tốt nghiệp          |mới tốt nghiệp         |9           |
+------------------------+-----------------------+------------+



3. Gán giá trị chuẩn hóa về cột chính cap_bac

In [16]:
#Ghi giá trị cap_bac chuẩn hóa về cột chính
#df = df.withColumn("cap_bac", col("cap_bac_code"))

#Xóa cột trung gian
#df = df.drop("cap_bac_standardized")
#df = df.drop("cap_bac_code")

df.show(10, truncate=False)

+-----------------------------------------------------------------------------------------------------------------------------------------+-----------+----------+-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+----------------------+-----------+---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

#Chuẩn hóa kinh_nghiem(ĐÃ XONG)

In [17]:
import re
from pyspark.sql.functions import udf, col
from pyspark.sql.types import DoubleType

def extract_exp_udf(text):
    if text is None:
        return None
    if not isinstance(text, str):
        # Nếu không phải chuỗi, chuyển sang chuỗi để xử lý
        text = str(text)
    text_lc = text.lower()
     # Các trường hợp biểu thị không yêu cầu hoặc không có kinh nghiệm
    if ('không yêu cầu' in text_lc or
        'chưa có kinh nghiệm' in text_lc or
        'không cần kinh nghiệm' in text_lc or
        'không phải là yếu tố bắt buộc' in text_lc or
        'không là yếu tố bắt buộc' in text_lc or
        'sẽ được đào tạo' in text_lc):
        return 0.0
    # Tìm tất cả số trong chuỗi, ví dụ "2 năm", "5-7 năm" -> [2], [5,7]
    numbers = [int(x) for x in re.findall(r'\d+', text)]
    # Nếu có từ "trên" (ví dụ: "trên 5 năm") và có số, trả về số đầu tiên (hoặc có thể +1)
    if 'trên' in text_lc and numbers:
        return float(numbers[0]+1)  # hoặc numbers[0]
    # Trường hợp "lên đến" hoặc tương tự thì lấy số đầu tiên
    if 'lên đến' in text_lc and numbers:
        return float(numbers[0])
    # Nếu chỉ có 1 số thì trả về số đó
    if len(numbers) == 1:
        return float(numbers[0])
    # Nếu có nhiều số (vd: "5-7 năm") thì lấy trung bình
    if len(numbers) > 1:
        return float(sum(numbers)/len(numbers))
    # Nếu không bắt được số nào, trả về None
    return None

# Đăng ký UDF trả về kiểu Double (float)
extract_exp = udf(extract_exp_udf, DoubleType())

# Áp dụng cho DataFrame pyspark
df = df.withColumn("exp_year", extract_exp(col("kinh_nghiem")))

# Đếm giá trị null trong exp_year
exp_missing = df.filter(col("exp_year").isNull()).count()
print(f"Số lượng giá trị thiếu trong exp_year: {exp_missing}")

if exp_missing == 0:
    print("Cột exp_year đã đầy đủ dữ liệu (không có missing value).")
else:
    print("Cột exp_year vẫn còn giá trị thiếu.")

# Xem giá trị unique của cột exp_year (lấy ví dụ 10 giá trị đầu tiên)
df.select("exp_year").distinct().show(10)


Số lượng giá trị thiếu trong exp_year: 13
Cột exp_year vẫn còn giá trị thiếu.
+--------+
|exp_year|
+--------+
|     8.0|
|     0.0|
|     7.0|
|    11.5|
|     3.5|
|     9.5|
|    31.5|
|    12.5|
|     4.5|
|     6.5|
+--------+
only showing top 10 rows



Những dòng link bị sai dữ liệu => Loại

In [18]:
df.filter(col("exp_year").isNull()).select("kinh_nghiem").show(truncate=False)

+----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|kinh_nghiem                                                                                                                                                                                                                                                                                                                                                               

Xóa những giá trị kinh_nghiem sai dữ liệu

In [19]:
from pyspark.sql.functions import col

# Lọc những dòng mà exp_year KHÔNG bị thiếu (isNotNull)
df_clean = df.filter(col("exp_year").isNotNull())

In [20]:
print("Số dòng ban đầu:", df.count())
print("Số dòng sau khi loại dòng lỗi:", df_clean.count())

Số dòng ban đầu: 11290
Số dòng sau khi loại dòng lỗi: 11277


In [21]:
# Gán giá trị exp_year về cột chính kinh_nghiem
#df = df_clean.withColumn("kinh_nghiem", col("exp_year"))

#Xóa cột trung gian
#df = df.drop("exp_year")

df_clean.show(5, truncate=False)

+-----------------------------------------------------------------------------------------------------------------------------------------+-----------+----------+-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+----------------------+-----------+---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

Đếm tần suất giá trị của kinh nghiem

In [22]:
df_clean.select("exp_year").distinct().orderBy("exp_year").show(100, truncate=False)

+--------+
|exp_year|
+--------+
|0.0     |
|1.0     |
|1.5     |
|1.75    |
|2.0     |
|2.5     |
|3.0     |
|3.5     |
|4.0     |
|4.5     |
|5.0     |
|5.5     |
|6.0     |
|6.5     |
|7.0     |
|7.5     |
|8.0     |
|8.5     |
|9.0     |
|9.5     |
|10.0    |
|10.5    |
|11.0    |
|11.5    |
|12.0    |
|12.5    |
|13.0    |
|13.5    |
|14.0    |
|14.5    |
|15.0    |
|16.0    |
|17.0    |
|17.5    |
|18.5    |
|20.0    |
|21.0    |
|31.5    |
+--------+



In [23]:
# Tổng số dòng ban đầu
total_rows = df_clean.count()
print("Tổng số dòng df_clean:", total_rows)

# Tổng theo exp_year (có giá trị không null)
total_exp_year = df_clean.filter(col("exp_year").isNotNull()).groupBy("exp_year").count().agg({"count": "sum"}).collect()[0][0]
print("Tổng số dòng theo exp_year không null:", total_exp_year)

# Số dòng có exp_year null
null_exp_year = df_clean.filter(col("exp_year").isNull()).count()
print("Số dòng có exp_year = null:", null_exp_year)

print("Tổng cộng dòng có exp_year null và không null:", total_exp_year + null_exp_year)



Tổng số dòng df_clean: 11277
Tổng số dòng theo exp_year không null: 11277
Số dòng có exp_year = null: 0
Tổng cộng dòng có exp_year null và không null: 11277


Gán giá trị quy ước

In [24]:
#Đưa gán cột kinh_nghiệm về giá trị quy ước
from pyspark.sql.functions import when, col

df = df_clean.withColumn("kinh_nghiem_group",
     when(col("exp_year") == 0, 0)
    .when((col("exp_year") > 0) & (col("exp_year") < 2), 1)
    .when((col("exp_year") >= 2) & (col("exp_year") < 5), 2)
    .when((col("exp_year") >= 5) & (col("exp_year") < 10), 3)
    .when(col("exp_year") >= 10, 4)
    .otherwise(-1)
)
df.groupBy("kinh_nghiem_group").count().orderBy("kinh_nghiem_group").show()

+-----------------+-----+
|kinh_nghiem_group|count|
+-----------------+-----+
|                0|  733|
|                1| 2171|
|                2| 6420|
|                3| 1710|
|                4|  243|
+-----------------+-----+



#Chuẩn hóa nganh_nghe (Đã xong)

In [25]:
#Kiểm tra các loại ngành nghề trong df
df.select("nganh_nghe").distinct().show(truncate=False)
unique_count = df.select("nganh_nghe").distinct().count()
print(f"Số lượng giá trị ngành nghề riêng biệt: {unique_count}")

+-------------------------------------------------------------------------------------+
|nganh_nghe                                                                           |
+-------------------------------------------------------------------------------------+
|Bán hàng / Kinh doanh, Bán lẻ / Bán sỉ, Xuất nhập khẩu                               |
|Bất động sản, Nhân sự                                                                |
|CNTT - Phần mềm, Ngân hàng                                                           |
|Dịch vụ khách hàng, Thực phẩm & Đồ uống, Nhà hàng / Khách sạn                        |
|Tiếp thị / Marketing, Bán hàng / Kinh doanh, Dược phẩm/ Hóa Mỹ Phẩm                  |
|Tư vấn, Nhà hàng / Khách sạn, Dịch vụ khách hàng                                     |
|Biên phiên dịch, Dệt may / Da giày / Thời trang, Thu mua / Vật tư                    |
|Du lịch                                                                              |
|Y tế / Chăm sóc sức khỏe / Thẩm

In [26]:
from pyspark.sql.functions import udf, col, concat_ws
from pyspark.sql.types import ArrayType, StringType

def split_and_clean(s):
    if s is None:
        return []
    parts = []
    # Tách theo dấu ,
    for part in s.split(","):
        # Tách tiếp theo dấu /
        sub_parts = part.split("/")
        for sp in sub_parts:
            sp_clean = sp.strip().lower()  # bạn có thể bỏ .lower() nếu muốn giữ nguyên chữ hoa thường
            if sp_clean:
                parts.append(sp_clean)
    # Loại bỏ trùng lặp
    return list(set(parts))

split_and_clean_udf = udf(split_and_clean, ArrayType(StringType()))

# Áp dụng UDF để tạo cột array đã xử lý
df = df.withColumn("nganh_nghe_clean_array", split_and_clean_udf(col("nganh_nghe")))

# Nối lại thành chuỗi, phân cách dấu ', '
df = df.withColumn("nganh_nghe_clean", concat_ws(", ", col("nganh_nghe_clean_array")))

df.select("nganh_nghe_clean").show(truncate=False)

+------------------------------------------------------------------------------------------+
|nganh_nghe_clean                                                                          |
+------------------------------------------------------------------------------------------+
|sản xuất, vận hành sản xuất                                                               |
|vận chuyển, giao nhận, kho vận                                                            |
|bán hàng, biên phiên dịch, kinh doanh                                                     |
|thương mại điện tử                                                                        |
|đầu tư, tài chính, ngân hàng                                                              |
|da giày, tiếp thị trực tuyến, marketing, tiếp thị, thời trang, dệt may                    |
|hóa học, công nghệ sinh học, môi trường                                                   |
|dệt may, da giày, thời trang                                         

In [27]:
df = df.drop("nganh_nghe_clean_array", "nganh_nghe_array", "nganh_nghe_array_trim")
df.show(5, truncate=False)

+-----------------------------------------------------------------------------------------------------------------------------------------+-----------+----------+-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+----------------------+-----------+---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

#Chuẩn hóa mo_ta_cong_viec & yeu_cau_cong_viec


In [28]:
# loại bỏ các ký tự  /n * • + -
from pyspark.sql.functions import col, regexp_replace, trim, length, when

# Làm sạch cột "Mô tả công việc"
desc1 = col("mo_ta_cong_viec")
desc1 = regexp_replace(desc1, r"[\n\r\*\•\+\-\●\•\/\(\)\=\@\#\$\%\^\&\*\!\?\~\`\|\.\,\;\:\_\-\"\']+|\.{2,}|…|‘|’|“|”", "")
desc1 = trim(desc1)  # Loại bỏ khoảng trắng đầu/cuối
desc1 = when(length(desc1) == 0, "nan").otherwise(desc1) # Xử lý trường hợp rỗng

# Làm sạch cột "Yêu cầu công việc"
desc2 = col("yeu_cau_cong_viec")
desc2 = regexp_replace(desc2, r"[\n\r\*\•\+\-\●\•\/\(\)\=\@\#\$\%\^\&\*\!\?\~\`\|\.\,\;\:\_\-\"\']+|\.{2,}|…|‘|’|“|”", "")
desc2 = trim(desc2)
desc2 = when(length(desc2) == 0, "nan").otherwise(desc2)

In [29]:
# Gán lại vào DataFrame
df_clean = df.withColumn("mo_ta_cong_viec", desc1) \
                .withColumn("yeu_cau_cong_viec", desc2)

# Loại bỏ các khoảng trắng thừa (nhiều khoảng trắng thành một, và trim đầu cuối)
df_cleaned = df_clean.withColumn(
    "mo_ta_cong_viec",
    trim(regexp_replace(col("mo_ta_cong_viec"), "\\s+", " "))
)
df_cleaned = df_clean.withColumn(
    "yeu_cau_cong_viec",
    trim(regexp_replace(col("yeu_cau_cong_viec"), "\\s+", " "))
)
df_cleaned.show(20, truncate=False)

+-----------------------------------------------------------------------------------------------------------------------------------------+-----------+----------+-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+----------------------+-----------+---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

Xóa cụm lỗi đầu dòng (MÔ TẢ CÔNG VIỆC/ YÊU CẦU CÔNG VIỆC)

In [30]:
#Xóa cụm đầu trong dòng
from pyspark.sql.functions import regexp_replace, col, lower
# mo_ta_cong_viec
df = df.withColumn("mo_ta_cong_viec", lower(col("mo_ta_cong_viec")))
df = df.withColumn(
    "mo_ta_cong_viec_clean",
    regexp_replace(col("mo_ta_cong_viec"), r"^\s*mô tả công việc,\s*", "")
)
# yeu_cau_cong_viec
df = df.withColumn("yeu_cau_cong_viec", lower(col("yeu_cau_cong_viec")))
df = df.withColumn(
    "yeu_cau_cong_viec_clean",
    regexp_replace(col("yeu_cau_cong_viec"), r"^\s*yêu cầu công việc,\s*", "")
)

df.show(15, truncate=False)

+-----------------------------------------------------------------------------------------------------------------------------------------+-----------+----------+-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+----------------------+-----------+---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

Cài thư viện langdetect và dịch en

In [34]:
!pip install langid
!pip install langdetect

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 981.5/981.5 kB 17.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for langdetect: filename=langdetect-1.0.9-py3-none-any.whl size=993223 sha256=c298edfbf5b53bd53bc50af6a9dc6d0a2b6f57ca00772758c77bf6d3ebb72756
  Stored in directory: /root/.cache/pip/wheels/0a/f2/b2/e5ca405801e05eb7c8ed5b3b4bcf1fcabcd6272c167640072e
Successfully built langdetect


In [35]:
import pandas as pd
from collections import Counter
from langdetect import detect, DetectorFactory
import re

DetectorFactory.seed = 0  # Đảm bảo kết quả ổn định

# Lấy dữ liệu từ Spark
sample_df = df.select("yeu_cau_cong_viec_clean", "mo_ta_cong_viec_clean")
pandas_df = sample_df.toPandas().copy()

# Hàm detect ngôn ngữ bằng langdetect
def detect_lang(text):
    try:
        if not text or not str(text).strip():
            return "nan"
        text = str(text).strip()
        if len(text) < 10:
            return "short_text"
        return detect(text)
    except:
        return "error"

# Áp dụng hàm detect
pandas_df["language_yeu_cau"] = pandas_df["yeu_cau_cong_viec_clean"].apply(detect_lang)
pandas_df["language_mo_ta"] = pandas_df["mo_ta_cong_viec_clean"].apply(detect_lang)

# Thống kê
print("Ngôn ngữ trong cột `yeu_cau_cong_viec_clean`:")
for lang, count in Counter(pandas_df["language_yeu_cau"]).most_common():
    print(f" - {lang:<12}: {count} dòng")

print("\nNgôn ngữ trong cột `mo_ta_cong_viec_clean`:")
for lang, count in Counter(pandas_df["language_mo_ta"]).most_common():
    print(f" - {lang:<12}: {count} dòng")

# Gộp lại với df gốc
df_to_pd = df.toPandas()
df_to_pd["language_yeu_cau"] = pandas_df["language_yeu_cau"]
df_to_pd["language_mo_ta"] = pandas_df["language_mo_ta"]

# Lọc những dòng vừa VI vừa EN
df_vi_en_both = df_to_pd[
    df_to_pd["language_yeu_cau"].isin(["vi", "en"]) &
    df_to_pd["language_mo_ta"].isin(["vi", "en"])
]


Ngôn ngữ trong cột `yeu_cau_cong_viec_clean`:
 - vi          : 9199 dòng
 - en          : 2044 dòng
 - short_text  : 31 dòng
 - zh-cn       : 2 dòng
 - fr          : 1 dòng

Ngôn ngữ trong cột `mo_ta_cong_viec_clean`:
 - vi          : 9022 dòng
 - en          : 2017 dòng
 - short_text  : 235 dòng
 - zh-cn       : 2 dòng
 - fr          : 1 dòng


In [36]:
# ------- Bonus: Kiểm tra dòng bị nhận sai tiếng Việt nhưng có tiếng Trung --------
def contains_chinese(text):
    return bool(re.search(r'[\u4e00-\u9fff]', str(text)))

# Các dòng `yeu_cau_cong_viec_clean` và mo_ta_cong_viec_clean bị nhận nhầm là "vi" nhưng có tiếng Trung
a_mask_wrong_vi = (pandas_df["language_yeu_cau"] == "vi") & pandas_df["yeu_cau_cong_viec_clean"].apply(contains_chinese)
b_mask_wrong_vi = (pandas_df["language_mo_ta"] == "vi") & pandas_df["mo_ta_cong_viec_clean"].apply(contains_chinese)
c_mask_wrong_en = (pandas_df["language_yeu_cau"] == "en") & pandas_df["yeu_cau_cong_viec_clean"].apply(contains_chinese)
d_mask_wrong_en = (pandas_df["language_mo_ta"] == "en") & pandas_df["mo_ta_cong_viec_clean"].apply(contains_chinese)

print("\nCác dòng bị nhầm là tiếng Việt nhưng chứa ký tự Trung Quốc:")


Các dòng bị nhầm là tiếng Việt nhưng chứa ký tự Trung Quốc:


In [37]:
pandas_df[a_mask_wrong_vi].head(10)

,yeu_cau_cong_viec_clean,mo_ta_cong_viec_clean,language_yeu_cau,language_mo_ta
9,"trình độ đại học trở lên, tiếng trung 4 kỹ nă...",chuyên viên tuyển dụng: chủ yếu chịu trách nhi...,vi,vi
16,"大学以上, 中国4技能，良好的沟通, 知道办公室计算机, 使用招聘网站有2年的招...","biên dịch: dịch văn bản, dịch lời nói (phiên d...",vi,vi
364,"trình độ thpt trở lên., 高中以上, có kinh nghiệm,...",chịu tránh nhiệm vận hành an toàn theo các thô...,vi,vi
1572,"yêu cầu công việc:, học vấn : tốt nghiệp đại ...",vị trí: kỹ sư thiết kế neo đất ngành xây dựng...,vi,vi
1801,"工作要求, 1、男生优先；, 2、大专及以上学历，, 3、沟通能力良好，执行能力强，抗压能力...","1、统筹管理整个采购部门，对整体交付负责；, 2、重大交付异常的跟踪处理, 3、负责采购周期...",vi,vi
2302,+ tiếng trung yêu cầu 4 kĩ năng thành thạo : n...,chịu trách nhiệm hỗ trợ quản lý và lập kế hoạc...,vi,vi
2786,"工作要求和福利, 职位要求：, 语言要求： 精通中文（听、说、读、写），同时具备良好的越南语...",mô tả công việc nhân viên hỗ trợ kinh doanh mả...,vi,vi
4460,"tốt nghiệp đại học trở lên, ưu tiên chuyên ngà...",hoàn thành mục tiêu kinh doanh về mảng tư vấn ...,vi,vi
4827,"1. bachelor degree in mechanical engineering, ...",1.solve processrelated problems in mechanical ...,vi,vi
4937,"教育背景：土木工程、商务管理、工程管理、国际商务、法律或相关专业本科及以上学历。, 专业技能...","岗位职责, 我们正在寻找一位具备国际视野与实战经验的越南属地高级管理人员，带领公司在越南的业...",vi,vi


In [38]:
pandas_df[b_mask_wrong_vi].head(10)

,yeu_cau_cong_viec_clean,mo_ta_cong_viec_clean,language_yeu_cau,language_mo_ta
9,"trình độ đại học trở lên, tiếng trung 4 kỹ nă...",chuyên viên tuyển dụng: chủ yếu chịu trách nhi...,vi,vi
16,"大学以上, 中国4技能，良好的沟通, 知道办公室计算机, 使用招聘网站有2年的招...","biên dịch: dịch văn bản, dịch lời nói (phiên d...",vi,vi
364,"trình độ thpt trở lên., 高中以上, có kinh nghiệm,...",chịu tránh nhiệm vận hành an toàn theo các thô...,vi,vi
1801,"工作要求, 1、男生优先；, 2、大专及以上学历，, 3、沟通能力良好，执行能力强，抗压能力...","1、统筹管理整个采购部门，对整体交付负责；, 2、重大交付异常的跟踪处理, 3、负责采购周期...",vi,vi
2302,+ tiếng trung yêu cầu 4 kĩ năng thành thạo : n...,chịu trách nhiệm hỗ trợ quản lý và lập kế hoạc...,vi,vi
2786,"工作要求和福利, 职位要求：, 语言要求： 精通中文（听、说、读、写），同时具备良好的越南语...",mô tả công việc nhân viên hỗ trợ kinh doanh mả...,vi,vi
4460,"tốt nghiệp đại học trở lên, ưu tiên chuyên ngà...",hoàn thành mục tiêu kinh doanh về mảng tư vấn ...,vi,vi
4796,tốt nghiệp cao đẳng/đại học chuyên ngành kinh ...,1. quản lý hoạt động chăm sóc và hỗ trợ khách ...,vi,vi
4827,"1. bachelor degree in mechanical engineering, ...",1.solve processrelated problems in mechanical ...,vi,vi
4937,"教育背景：土木工程、商务管理、工程管理、国际商务、法律或相关专业本科及以上学历。, 专业技能...","岗位职责, 我们正在寻找一位具备国际视野与实战经验的越南属地高级管理人员，带领公司在越南的业...",vi,vi


In [39]:
pandas_df[c_mask_wrong_en].head(10)

,yeu_cau_cong_viec_clean,mo_ta_cong_viec_clean,language_yeu_cau,language_mo_ta
1681,"工作经验：5年以上工作经验，熟悉国际快递和供应链业务管理经验。, work experien...",顺丰作为全球领先的综合物流服务提供商，在国际物流领域持续拓展业务版图。现因柬埔寨业务发展需要...,en,en
2342,"qualifications, education: , 本科及以上学历，工程管理、土木工程...",1. 项目协调与进度管理project coordination & progress ma...,en,en
3323,any certification on apparel management will b...,operating target setup as per allocated machin...,en,en
5510,"requirements, education, bachelor or above deg...","job summary, manage all suppliers’ quality and...",en,en
8322,"- no-experienced is okay, have experienced is ...",#name?,en,short_text
8341,1. 年龄: 28-35 岁， 形象气质佳，\n2. 学历: 人专及以上\n3. 有独立开发...,每日核心工作:(完成每项工作花费时间占全部工作时间的百分比)\n1. 客户开发与沟通 (40...,en,en
8933,• 至少 3 年建筑材料行业的销售或工程经验。\n• 较强的沟通和人际交往能力。\n• 熟练...,• 识别和开发建筑行业内的新市场机会。\n• 与潜在客户进行陌生电话联系，以建立新的业务关系...,en,en
9023,1. college degree or above; majors in electron...,key responsibilities:\n1. responsible for deve...,en,en
9503,1. 年龄: 28-35 岁， 形象气质佳，\n2. 学历: 人专及以上\n3. 本科及以上...,1. 负责日常会计凭证的录入与账务处理， 确保账目清晰、 准确；\n2. 编制月度、 季度、...,en,en
9613,• 大学学历以上，仅限财务相关专业。\n- university degree or abo...,• 负责公司海防总部财务会计工作，包括总账稽核、纳税申报、合并报表编制。\n• respon...,en,en


In [40]:
pandas_df[d_mask_wrong_en].head(10)

,yeu_cau_cong_viec_clean,mo_ta_cong_viec_clean,language_yeu_cau,language_mo_ta
1681,"工作经验：5年以上工作经验，熟悉国际快递和供应链业务管理经验。, work experien...",顺丰作为全球领先的综合物流服务提供商，在国际物流领域持续拓展业务版图。现因柬埔寨业务发展需要...,en,en
2342,"qualifications, education: , 本科及以上学历，工程管理、土木工程...",1. 项目协调与进度管理project coordination & progress ma...,en,en
3323,any certification on apparel management will b...,operating target setup as per allocated machin...,en,en
8341,1. 年龄: 28-35 岁， 形象气质佳，\n2. 学历: 人专及以上\n3. 有独立开发...,每日核心工作:(完成每项工作花费时间占全部工作时间的百分比)\n1. 客户开发与沟通 (40...,en,en
8933,• 至少 3 年建筑材料行业的销售或工程经验。\n• 较强的沟通和人际交往能力。\n• 熟练...,• 识别和开发建筑行业内的新市场机会。\n• 与潜在客户进行陌生电话联系，以建立新的业务关系...,en,en
9023,1. college degree or above; majors in electron...,key responsibilities:\n1. responsible for deve...,en,en
9503,1. 年龄: 28-35 岁， 形象气质佳，\n2. 学历: 人专及以上\n3. 本科及以上...,1. 负责日常会计凭证的录入与账务处理， 确保账目清晰、 准确；\n2. 编制月度、 季度、...,en,en
9613,• 大学学历以上，仅限财务相关专业。\n- university degree or abo...,• 负责公司海防总部财务会计工作，包括总账稽核、纳税申报、合并报表编制。\n• respon...,en,en
9869,10年會計長經驗\n中文或英文流利\n10 years chief accountant e...,• 財務報告與帳務合規：\n• 確保所有會計記錄、憑證和報表符合越南會計準則（vas）及相關...,en,en
11271,1. education\nbachelor’s degree or above in a ...,your role / what will you do?\n1. policy resea...,en,en


In [41]:
# lọc lại
# Gán "nan" cho language_yeu_cau nếu là tiếng Việt nhưng chứa tiếng Trung
df_vi_en_both.loc[a_mask_wrong_vi, "language_yeu_cau"] = "nan"

# Gán "nan" cho language_mo_ta nếu là tiếng Việt nhưng chứa tiếng Trung
df_vi_en_both.loc[b_mask_wrong_vi, "language_mo_ta"] = "nan"

# Gán "nan" cho language_yeu_cau nếu là tiếng Anh nhưng chứa tiếng Trung
df_vi_en_both.loc[c_mask_wrong_en, "language_yeu_cau"] = "nan"

# Gán "nan" cho language_mo_ta nếu là tiếng Anh nhưng chứa tiếng Trung
df_vi_en_both.loc[d_mask_wrong_en, "language_mo_ta"] = "nan"


# Lọc những dòng đúng ngôn ngữ vi/en và không bị dính sai
df_vi_en_both_clean = df_vi_en_both[
    df_vi_en_both["language_yeu_cau"].isin(["vi", "en"]) &
    df_vi_en_both["language_mo_ta"].isin(["vi", "en"])
]
print("Tổng số dòng ban đầu : ", len(df_to_pd))
print("Trước khi loại bỏ các ký tự trung : ",len(df_vi_en_both))
print("Sau khi loại bỏ các ký tự trung : ",len(df_vi_en_both_clean))

Tổng số dòng ban đầu :  11277
Trước khi loại bỏ các ký tự trung :  11014
Sau khi loại bỏ các ký tự trung :  10977


In [42]:
print(df_vi_en_both_clean.columns)

Index(['ten_cong_viec', 'khu_vuc', 'ngay_dang', 'link', 'cap_bac',
       'kinh_nghiem', 'mo_ta_cong_viec', 'yeu_cau_cong_viec', 'nganh_nghe',
       'ngay_dang_clean', 'ngay_dang_date', 'is_valid_link',
       'cap_bac_standardized', 'cap_bac_code', 'exp_year', 'kinh_nghiem_group',
       'nganh_nghe_clean', 'mo_ta_cong_viec_clean', 'yeu_cau_cong_viec_clean',
       'language_yeu_cau', 'language_mo_ta'],
      dtype='object')


In [43]:
df_vi_en_both_clean.head(16) # xóa luôn cả id luôn =)

,ten_cong_viec,khu_vuc,ngay_dang,link,cap_bac,kinh_nghiem,mo_ta_cong_viec,yeu_cau_cong_viec,nganh_nghe,ngay_dang_clean,...,is_valid_link,cap_bac_standardized,cap_bac_code,exp_year,kinh_nghiem_group,nganh_nghe_clean,mo_ta_cong_viec_clean,yeu_cau_cong_viec_clean,language_yeu_cau,language_mo_ta
0,Chuyên viên kế hoạch sản xuất-Đồng Nai,Đồng Nai,18-07-2025,https://careerviet.vn/vi/tim-viec-lam/chuyen-v...,nhân viên,2 - 3 Năm,"mô tả công việc, làm việc bộ phận bán hàng: t...","yêu cầu công việc, nam/nữ, từ 28 – 40 tuổi, ...",Sản xuất / Vận hành sản xuất,18-07-2025,...,True,nhân viên,7,2.5,2,"sản xuất, vận hành sản xuất","làm việc bộ phận bán hàng: thông tin đơn hàng,...","nam/nữ, từ 28 – 40 tuổi, tốt nghiệp đại học t...",vi,vi
1,Sales Freehand,Hồ Chí Minh,18-07-2025,https://careerviet.vn/vi/tim-viec-lam/sales-fr...,nhân viên,1 - 3 Năm,"mô tả công việc, · focusing on sales performan...","yêu cầu công việc, · min 1 3 years of e...",Vận chuyển / Giao nhận / Kho vận,18-07-2025,...,True,nhân viên,7,2.0,2,"vận chuyển, giao nhận, kho vận",· focusing on sales performance to achieve tea...,· min 1 3 years of experience in relate...,en,en
2,[Tuyển Gấp] Phiên Dịch Tiếng Trung và Xúc Tiến...,Hà Nội,18-07-2025,https://careerviet.vn/vi/tim-viec-lam/tuyen-ga...,trưởng nhóm / giám sát,5 - 15 Năm,"mô tả công việc, mô tả công việc, chức danh: p...","yêu cầu công việc, êu cầu công việc, nam/nữ d...","Bán hàng / Kinh doanh, Biên phiên dịch",18-07-2025,...,True,trưởng nhóm / giám sát,6,10.0,4,"bán hàng, biên phiên dịch, kinh doanh","mô tả công việc, chức danh: phiên dịch tiếng t...","êu cầu công việc, nam/nữ dưới 40 tuổi;, kinh...",vi,vi
3,Sourcing Leader (Furniture),Hồ Chí Minh,18-07-2025,https://careerviet.vn/vi/tim-viec-lam/sourcing...,trưởng nhóm / giám sát,3 - 7 Năm,"mô tả công việc, founded in 2009 in orange cou...","yêu cầu công việc, requirements, education, ba...",Thương mại điện tử,18-07-2025,...,True,trưởng nhóm / giám sát,6,5.0,3,thương mại điện tử,"founded in 2009 in orange county, california, ...","requirements, education, bachelor‘s degree in ...",en,en
4,"Chuyên viên Kinh doanh ngoại hối, phái sinh -...",Hồ Chí Minh,18-07-2025,https://careerviet.vn/vi/tim-viec-lam/chuyen-v...,nhân viên,Trên 2 Năm,"mô tả công việc, trực tiếp tư vấn thị trường, ...","yêu cầu công việc, cử nhân chính quy trở lên c...","Ngân hàng, Tài chính / Đầu tư",18-07-2025,...,True,nhân viên,7,3.0,2,"đầu tư, tài chính, ngân hàng","trực tiếp tư vấn thị trường, giao dịch ngoại t...",cử nhân chính quy trở lên chuyên ngành kinh tế...,vi,vi
5,Marketing Manager (Up 5000S),Hà Nội,18-07-2025,https://careerviet.vn/vi/tim-viec-lam/marketin...,quản lý,Trên 5 Năm,"mô tả công việc, vị trí: marketing manager (ph...","yêu cầu công việc, trình độ học vấn: tốt nghi...","Tiếp thị / Marketing, Tiếp thị trực tuyến, Dệt...",18-07-2025,...,True,quản lý,5,6.0,3,"da giày, tiếp thị trực tuyến, marketing, tiếp ...",vị trí: marketing manager (phòng marketing 90 ...,trình độ học vấn: tốt nghiệp đại học chuyên ng...,vi,vi
6,Nhân viên Thẩm Xét,Hà Nội,18-07-2025,https://careerviet.vn/vi/tim-viec-lam/nhan-vie...,nhân viên,Trên 1 Năm,"mô tả công việc, tiếp nhận và xem xét yêu cầu ...","yêu cầu công việc, tốt nghiệp đại học các chuy...","Công nghệ sinh học, Môi trường, Hóa học",18-07-2025,...,True,nhân viên,7,2.0,2,"hóa học, công nghệ sinh học, môi trường",tiếp nhận và xem xét yêu cầu đầu vào đối với n...,tốt nghiệp đại học các chuyên ngành: tài nguyê...,vi,vi
7,[AN KHÁNH - HOÀI ĐỨC - HÀ NỘI] Nhân Viên Kho,Hà Nội,18-07-2025,https://careerviet.vn/vi/tim-viec-lam/an-khanh...,nhân viên,Trên 1 Năm,"mô tả công việc, * nhân viên kho vận:, nhập ...","yêu cầu công việc, nam, tuổi từ 20 35, sức k...",Dệt may / Da giày / Thời trang,18-07-2025,...,True,nhân viên,7,2.0,2,"dệt may, da giày, thời trang","* nhân viên kho vận:, nhập xuất hàng, kiểm đ...","nam, tuổi từ 20 35, sức khỏe tốt., trung thự...",vi,vi
8,Trưởng nhóm Bán sản phẩm cho vay tiểu thương/h...,Hà Nội,18-07-2025,https://careerviet.vn/vi/tim-viec-lam/tru

In [44]:
df = df_vi_en_both_clean.reset_index(drop=True)
df.head(16)

,ten_cong_viec,khu_vuc,ngay_dang,link,cap_bac,kinh_nghiem,mo_ta_cong_viec,yeu_cau_cong_viec,nganh_nghe,ngay_dang_clean,...,is_valid_link,cap_bac_standardized,cap_bac_code,exp_year,kinh_nghiem_group,nganh_nghe_clean,mo_ta_cong_viec_clean,yeu_cau_cong_viec_clean,language_yeu_cau,language_mo_ta
0,Chuyên viên kế hoạch sản xuất-Đồng Nai,Đồng Nai,18-07-2025,https://careerviet.vn/vi/tim-viec-lam/chuyen-v...,nhân viên,2 - 3 Năm,"mô tả công việc, làm việc bộ phận bán hàng: t...","yêu cầu công việc, nam/nữ, từ 28 – 40 tuổi, ...",Sản xuất / Vận hành sản xuất,18-07-2025,...,True,nhân viên,7,2.5,2,"sản xuất, vận hành sản xuất","làm việc bộ phận bán hàng: thông tin đơn hàng,...","nam/nữ, từ 28 – 40 tuổi, tốt nghiệp đại học t...",vi,vi
1,Sales Freehand,Hồ Chí Minh,18-07-2025,https://careerviet.vn/vi/tim-viec-lam/sales-fr...,nhân viên,1 - 3 Năm,"mô tả công việc, · focusing on sales performan...","yêu cầu công việc, · min 1 3 years of e...",Vận chuyển / Giao nhận / Kho vận,18-07-2025,...,True,nhân viên,7,2.0,2,"vận chuyển, giao nhận, kho vận",· focusing on sales performance to achieve tea...,· min 1 3 years of experience in relate...,en,en
2,[Tuyển Gấp] Phiên Dịch Tiếng Trung và Xúc Tiến...,Hà Nội,18-07-2025,https://careerviet.vn/vi/tim-viec-lam/tuyen-ga...,trưởng nhóm / giám sát,5 - 15 Năm,"mô tả công việc, mô tả công việc, chức danh: p...","yêu cầu công việc, êu cầu công việc, nam/nữ d...","Bán hàng / Kinh doanh, Biên phiên dịch",18-07-2025,...,True,trưởng nhóm / giám sát,6,10.0,4,"bán hàng, biên phiên dịch, kinh doanh","mô tả công việc, chức danh: phiên dịch tiếng t...","êu cầu công việc, nam/nữ dưới 40 tuổi;, kinh...",vi,vi
3,Sourcing Leader (Furniture),Hồ Chí Minh,18-07-2025,https://careerviet.vn/vi/tim-viec-lam/sourcing...,trưởng nhóm / giám sát,3 - 7 Năm,"mô tả công việc, founded in 2009 in orange cou...","yêu cầu công việc, requirements, education, ba...",Thương mại điện tử,18-07-2025,...,True,trưởng nhóm / giám sát,6,5.0,3,thương mại điện tử,"founded in 2009 in orange county, california, ...","requirements, education, bachelor‘s degree in ...",en,en
4,"Chuyên viên Kinh doanh ngoại hối, phái sinh -...",Hồ Chí Minh,18-07-2025,https://careerviet.vn/vi/tim-viec-lam/chuyen-v...,nhân viên,Trên 2 Năm,"mô tả công việc, trực tiếp tư vấn thị trường, ...","yêu cầu công việc, cử nhân chính quy trở lên c...","Ngân hàng, Tài chính / Đầu tư",18-07-2025,...,True,nhân viên,7,3.0,2,"đầu tư, tài chính, ngân hàng","trực tiếp tư vấn thị trường, giao dịch ngoại t...",cử nhân chính quy trở lên chuyên ngành kinh tế...,vi,vi
5,Marketing Manager (Up 5000S),Hà Nội,18-07-2025,https://careerviet.vn/vi/tim-viec-lam/marketin...,quản lý,Trên 5 Năm,"mô tả công việc, vị trí: marketing manager (ph...","yêu cầu công việc, trình độ học vấn: tốt nghi...","Tiếp thị / Marketing, Tiếp thị trực tuyến, Dệt...",18-07-2025,...,True,quản lý,5,6.0,3,"da giày, tiếp thị trực tuyến, marketing, tiếp ...",vị trí: marketing manager (phòng marketing 90 ...,trình độ học vấn: tốt nghiệp đại học chuyên ng...,vi,vi
6,Nhân viên Thẩm Xét,Hà Nội,18-07-2025,https://careerviet.vn/vi/tim-viec-lam/nhan-vie...,nhân viên,Trên 1 Năm,"mô tả công việc, tiếp nhận và xem xét yêu cầu ...","yêu cầu công việc, tốt nghiệp đại học các chuy...","Công nghệ sinh học, Môi trường, Hóa học",18-07-2025,...,True,nhân viên,7,2.0,2,"hóa học, công nghệ sinh học, môi trường",tiếp nhận và xem xét yêu cầu đầu vào đối với n...,tốt nghiệp đại học các chuyên ngành: tài nguyê...,vi,vi
7,[AN KHÁNH - HOÀI ĐỨC - HÀ NỘI] Nhân Viên Kho,Hà Nội,18-07-2025,https://careerviet.vn/vi/tim-viec-lam/an-khanh...,nhân viên,Trên 1 Năm,"mô tả công việc, * nhân viên kho vận:, nhập ...","yêu cầu công việc, nam, tuổi từ 20 35, sức k...",Dệt may / Da giày / Thời trang,18-07-2025,...,True,nhân viên,7,2.0,2,"dệt may, da giày, thời trang","* nhân viên kho vận:, nhập xuất hàng, kiểm đ...","nam, tuổi từ 20 35, sức khỏe tốt., trung thự...",vi,vi
8,Trưởng nhóm Bán sản phẩm cho vay tiểu thương/h...,Hà Nội,18-07-2025,https://careerviet.vn/vi/tim-viec-lam/tru

# DỊCH

In [45]:
temp = df.copy()

In [47]:
!pip install googletrans==4.0.0-rc1
!pip install deep-translator

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.3/42.3 kB 2.9 MB/s eta 0:00:00


In [48]:
from langdetect import detect
from deep_translator import GoogleTranslator
import re
from tqdm import tqdm
from concurrent.futures import ThreadPoolExecutor, as_completed

# Hàm nhận diện text có phải tiếng Việt hay không (dựa trên dấu)
def looks_like_vietnamese(text):
    return bool(re.search(r"[àáạảãâầấậẩẫăằắặẳẵèéẹẻẽêềếệểễ"
                          r"ìíịỉĩòóọỏõôồốộổỗơờớợởỡ"
                          r"ùúụủũưừứựửữỳýỵỷỹđ]", text.lower()))

# Hàm dịch
def translate_multilang(text):
    if not text or not str(text).strip():
        return "nan"

    text = str(text).strip()

    try:
        lang = detect(text).lower()
    except:
        lang = ""

    # Nếu detect sai nhưng có dấu tiếng Việt → coi là tiếng Việt
    if lang not in ["vi", "en"] and not looks_like_vietnamese(text):
        return "nan"

    # Nếu là tiếng Anh → giữ nguyên
    if lang == "en":
        return text

    # Nếu là tiếng Việt → dịch
    try:
        return GoogleTranslator(source='auto', target='en').translate(text)
    except:
        return "nan"

# Hàm dịch đa luồng
def parallel_translate(text_list, max_workers=10, desc="Đang dịch..."):
    results = [None] * len(text_list)
    with ThreadPoolExecutor(max_workers=max_workers) as executor:
        futures = {executor.submit(translate_multilang, text): idx for idx, text in enumerate(text_list)}
        for future in tqdm(as_completed(futures), total=len(futures), desc=desc):
            idx = futures[future]
            try:
                results[idx] = future.result()
            except:
                results[idx] = "nan"
    return results

In [49]:
# Test dịch 5 dòng đầu tiên
test_texts = temp["yeu_cau_cong_viec_clean"].head(5).tolist()
translated_test = parallel_translate(test_texts, max_workers=3)

for original, translated in zip(test_texts, translated_test):
    print("-----")
    print(f"Original  : {original}")
    print(f"Translated: {translated}")
test_texts = temp["mo_ta_cong_viec_clean"].head(5).tolist()
translated_test = parallel_translate(test_texts, max_workers=3)

for original, translated in zip(test_texts, translated_test):
    print("-----")
    print(f"Original  : {original}")
    print(f"Translated: {translated}")

Đang dịch...: 100%|██████████| 5/5 [00:00<00:00, 51.83it/s]


-----
Original  : nam/nữ, từ 28 – 40 tuổi,  tốt nghiệp đại học trở lên các ngành: kinh tế, thống kê, kế hoạch,  ít nhất 2 năm kinh nghiệm ở vị trí tương tự,  sử dụng thành thạo kỹ năng tin học văn phòng, đặc biệt là excel; google sheet. ưu tiên ứng viên đã từng sử dụng các phần mềm quản lý sản xuất.,  hiểu về các quy trình sản xuất, kế hoạch sản xuất,  kỹ năng lập kế hoạch ,  biết cách thu thập và phân tích dữ liệu,  kỹ năng báo cáo và hiểu các chỉ số,  tiếp thu và xử lý công việc nhanh.,  kỹ năng giao tiếp và truyền đạt thông tin tốt.,  cẩn thận, tỉ mỉ, có trách nhiệm trong công việc.
Translated: Male/female, from 28 to 40 years old, graduated from university or higher in the fields of economics, statistics, plan, at least 2 years of experience in the same position, proficient use of office information skills skills, especially Excel; Google Sheet. Priority is given to candidates who have used production management software, understand production processes, production plans, planning 

Đang dịch...: 100%|██████████| 5/5 [00:01<00:00,  4.17it/s]

-----
Original  : làm việc bộ phận bán hàng: thông tin đơn hàng, khả năng đáp ứng, sự cố khi thực hiện đơn hàng.,  làm việc với phòng mua hàng. thông tin kế hoạch hàng về, xác nhận về, báo cáo chât lượng và dịch vụ hàng hóa nhà cung cấp.,  báo cáo dự kiến hoàn thành các đơn hàng cho gđnm, cảnh báo sản lượng vượt mức thiết kế, tham mưu giải pháp cho gđnm ra quyết định.,  rà soát tồn kho từ báo cáo tồn kho của tn kho và phối hợp với các bộ phận liên quan đảm bảo tồn kho an toàn đáp ứng nhu cầu kinh doanh.,  chịu trách nhiệm thu thập, tổng hợp và điền các dữ liệu báo cáo hàng ngày dưới sự chỉ đạo của gđnm và hướng dẫn của phòng khsx ho.,  hỗ trợ gđnm phối hợp với nhân sự phòng khsx ho triển khai thực hiện, tuân thủ các quy trình, quy định, văn bản tại nhà máy từ ho.,  báo cáo cho gđnm khi thấy chỉ số hiệu suất máy, năng suất lao động, thấp hơn chỉ số kế hoạch.,  thực hiện các công việc khác theo chỉ đạo của gđnm.
Translated: Sales work: Orders information, ability to meet, incidents when 

In [50]:
# Dịch toàn bộ cột "yeu_cau_cong_viec_clean"
test_texts = temp["yeu_cau_cong_viec_clean"].tolist()
translated_test = parallel_translate(test_texts, max_workers=20)
temp["yeu_cau_cong_viec_en"] = translated_test  # Lưu kết quả vào cột mới

# Dịch toàn bộ cột "mo_ta_cong_viec_clean"
test_texts = temp["mo_ta_cong_viec_clean"].tolist()
translated_test = parallel_translate(test_texts, max_workers=20)
temp["mo_ta_cong_viec_en"] = translated_test  # Lưu kết quả vào cột mới

Đang dịch...: 100%|██████████| 10977/10977 [05:44<00:00, 31.84it/s]


In [51]:
temp.head(20)

,ten_cong_viec,khu_vuc,ngay_dang,link,cap_bac,kinh_nghiem,mo_ta_cong_viec,yeu_cau_cong_viec,nganh_nghe,ngay_dang_clean,...,cap_bac_code,exp_year,kinh_nghiem_group,nganh_nghe_clean,mo_ta_cong_viec_clean,yeu_cau_cong_viec_clean,language_yeu_cau,language_mo_ta,yeu_cau_cong_viec_en,mo_ta_cong_viec_en
0,Chuyên viên kế hoạch sản xuất-Đồng Nai,Đồng Nai,18-07-2025,https://careerviet.vn/vi/tim-viec-lam/chuyen-v...,nhân viên,2 - 3 Năm,"mô tả công việc, làm việc bộ phận bán hàng: t...","yêu cầu công việc, nam/nữ, từ 28 – 40 tuổi, ...",Sản xuất / Vận hành sản xuất,18-07-2025,...,7,2.5,2,"sản xuất, vận hành sản xuất","làm việc bộ phận bán hàng: thông tin đơn hàng,...","nam/nữ, từ 28 – 40 tuổi, tốt nghiệp đại học t...",vi,vi,"Male/female, from 28 to 40 years old, graduate...","Sales work: Orders information, ability to mee..."
1,Sales Freehand,Hồ Chí Minh,18-07-2025,https://careerviet.vn/vi/tim-viec-lam/sales-fr...,nhân viên,1 - 3 Năm,"mô tả công việc, · focusing on sales performan...","yêu cầu công việc, · min 1 3 years of e...",Vận chuyển / Giao nhận / Kho vận,18-07-2025,...,7,2.0,2,"vận chuyển, giao nhận, kho vận",· focusing on sales performance to achieve tea...,· min 1 3 years of experience in relate...,en,en,· min 1 3 years of experience in relate...,· focusing on sales performance to achieve tea...
2,[Tuyển Gấp] Phiên Dịch Tiếng Trung và Xúc Tiến...,Hà Nội,18-07-2025,https://careerviet.vn/vi/tim-viec-lam/tuyen-ga...,trưởng nhóm / giám sát,5 - 15 Năm,"mô tả công việc, mô tả công việc, chức danh: p...","yêu cầu công việc, êu cầu công việc, nam/nữ d...","Bán hàng / Kinh doanh, Biên phiên dịch",18-07-2025,...,6,10.0,4,"bán hàng, biên phiên dịch, kinh doanh","mô tả công việc, chức danh: phiên dịch tiếng t...","êu cầu công việc, nam/nữ dưới 40 tuổi;, kinh...",vi,vi,"Work requirements, men/women under 40 years ol...","Job description, title: Chinese translation an..."
3,Sourcing Leader (Furniture),Hồ Chí Minh,18-07-2025,https://careerviet.vn/vi/tim-viec-lam/sourcing...,trưởng nhóm / giám sát,3 - 7 Năm,"mô tả công việc, founded in 2009 in orange cou...","yêu cầu công việc, requirements, education, ba...",Thương mại điện tử,18-07-2025,...,6,5.0,3,thương mại điện tử,"founded in 2009 in orange county, california, ...","requirements, education, bachelor‘s degree in ...",en,en,"requirements, education, bachelor‘s degree in ...","founded in 2009 in orange county, california, ..."
4,"Chuyên viên Kinh doanh ngoại hối, phái sinh -...",Hồ Chí Minh,18-07-2025,https://careerviet.vn/vi/tim-viec-lam/chuyen-v...,nhân viên,Trên 2 Năm,"mô tả công việc, trực tiếp tư vấn thị trường, ...","yêu cầu công việc, cử nhân chính quy trở lên c...","Ngân hàng, Tài chính / Đầu tư",18-07-2025,...,7,3.0,2,"đầu tư, tài chính, ngân hàng","trực tiếp tư vấn thị trường, giao dịch ngoại t...",cử nhân chính quy trở lên chuyên ngành kinh tế...,vi,vi,"Bachelor of regular or higher in economics, ba...","Direct market consultancy, foreign currency tr..."
5,Marketing Manager (Up 5000S),Hà Nội,18-07-2025,https://careerviet.vn/vi/tim-viec-lam/marketin...,quản lý,Trên 5 Năm,"mô tả công việc, vị trí: marketing manager (ph...","yêu cầu công việc, trình độ học vấn: tốt nghi...","Tiếp thị / Marketing, Tiếp thị trực tuyến, Dệt...",18-07-2025,...,5,6.0,3,"da giày, tiếp thị trực tuyến, marketing, tiếp ...",vị trí: marketing manager (phòng marketing 90 ...,trình độ học vấn: tốt nghiệp đại học chuyên ng...,vi,vi,Education level: Graduated from university maj...,Location: Marketing Manager (Marketing Departm...
6,Nhân viên Thẩm Xét,Hà Nội,18-07-2025,https://careerviet.vn/vi/tim-viec-lam/nhan-vie...,nhân viên,Trên 1 Năm,"mô tả công việc, tiếp nhận và xem xét yêu cầu ...","yêu cầu công việc, tốt nghiệp đại học các chuy...","Công nghệ sinh học, Môi trường, Hóa học",18-07-2025,...,7,2.0,2,"hóa học, công nghệ sinh học, môi trường",tiếp nhận và xem xét yêu cầu đầu vào đối với n...,tốt nghiệp đại học các chuyên ngành: tài nguyê...,vi,vi,Graduated from university in specialized major...,Receiving and cons

In [52]:
print(list(temp.columns))

['ten_cong_viec', 'khu_vuc', 'ngay_dang', 'link', 'cap_bac', 'kinh_nghiem', 'mo_ta_cong_viec', 'yeu_cau_cong_viec', 'nganh_nghe', 'ngay_dang_clean', 'ngay_dang_date', 'is_valid_link', 'cap_bac_standardized', 'cap_bac_code', 'exp_year', 'kinh_nghiem_group', 'nganh_nghe_clean', 'mo_ta_cong_viec_clean', 'yeu_cau_cong_viec_clean', 'language_yeu_cau', 'language_mo_ta', 'yeu_cau_cong_viec_en', 'mo_ta_cong_viec_en']


In [53]:
# Danh sách các cột cần lấy
selected_columns = [
    'ten_cong_viec',
    'khu_vuc',
    'ngay_dang_clean',
    'ngay_dang_date',
    'link',
    'cap_bac_standardized',
    'cap_bac_code',
    'exp_year',
    'kinh_nghiem_group',
    'nganh_nghe_clean',
    'mo_ta_cong_viec_clean',
    'yeu_cau_cong_viec_clean',
    'yeu_cau_cong_viec_en',
    'mo_ta_cong_viec_en'
]

# Lấy ra bản chỉ gồm các cột đó
final_df = temp[selected_columns].copy()

In [54]:
from google.colab import files
final_df.to_excel("Tien_xu_ly_finish_dich.xlsx", index=False)
files.download("Tien_xu_ly_finish_dich.xlsx")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [56]:
import pandas as pd

# Giả sử df là DataFrame của bạn
final_df.to_parquet('/content/drive/MyDrive/tien_xu_ly_jobs.parquet',  engine='pyarrow', compression='snappy')